## [Parameters](https://ardupilot.org/dev/docs/mavlink-get-set-params.html)

In [1]:
from pymavlink import mavutil
import subprocess
import time
import os

Kill related process

In [2]:
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")

Launch Copter (ardupilot)

In [3]:
sim_vehicle_path = os.path.expanduser("~/ardupilot/Tools/autotest/sim_vehicle.py")
vehicle_cmd = f"python3 {sim_vehicle_path} -v ArduCopter" 
# Open a new terminal and run the command
subprocess.Popen(["gnome-terminal", "--", "bash", "-c", f"{vehicle_cmd}; exec bash"])

<Popen: returncode: None args: ['gnome-terminal', '--', 'bash', '-c', 'pytho...>

## Conect to the device

In [4]:
conn = mavutil.mavlink_connection('udp:127.0.0.1:14550')

conn.wait_heartbeat()
print("Heartbeat received. Connected to the vehicle.")

Heartbeat received. Connected to the vehicle.


## Retrieving All Parameters

send a [PARAM_REQUEST_LIST](https://mavlink.io/en/messages/common.html#PARAM_REQUEST_LIST)

In [5]:
conn.mav.param_request_list_send(conn.target_system, conn.target_component)
heraing_time=1
start_time = time.time()  # Get the current time
print(f"hearing values for {heraing_time} sec")
while time.time() - start_time < heraing_time:  # Run for 0.01 seconds
    msg=conn.recv_match('PARAM_VALUE',blocking=True)
    if msg is not None:
        print(msg)


hearing values for 1 sec
PARAM_VALUE {param_id : FORMAT_VERSION, param_value : 120.0, param_type : 4, param_count : 1390, param_index : 0}
PARAM_VALUE {param_id : FORMAT_VERSION, param_value : 120.0, param_type : 4, param_count : 1390, param_index : 0}
PARAM_VALUE {param_id : SYSID_THISMAV, param_value : 1.0, param_type : 4, param_count : 1390, param_index : 1}
PARAM_VALUE {param_id : SYSID_THISMAV, param_value : 1.0, param_type : 4, param_count : 1390, param_index : 1}
PARAM_VALUE {param_id : SYSID_MYGCS, param_value : 255.0, param_type : 4, param_count : 1390, param_index : 2}
PARAM_VALUE {param_id : SYSID_MYGCS, param_value : 255.0, param_type : 4, param_count : 1390, param_index : 2}
PARAM_VALUE {param_id : PILOT_THR_FILT, param_value : 0.0, param_type : 9, param_count : 1390, param_index : 3}
PARAM_VALUE {param_id : PILOT_THR_FILT, param_value : 0.0, param_type : 9, param_count : 1390, param_index : 3}
PARAM_VALUE {param_id : PILOT_TKOFF_ALT, param_value : 0.0, param_type : 9, par

recieve the parameter list

Rarameter type

### Ardupilot supports ony the folowing paramter types
* MAV_PARAM_TYPE_INT8 (2)
* MAV_PARAM_TYPE_INT16 (4)
* MAV_PARAM_TYPE_INT32 (6)
* MAV_PARAM_TYPE_REAL32 (9) (aka float)

[Full list of Mavlink parameter types](https://mavlink.io/en/messages/common.html#MAV_PARAM_TYPE)

## Retrieving a single Parameter

#### Send a [param_request_read](https://mavlink.io/en/messages/common.html#PARAM_REQUEST_READ)

In [11]:
param_id = "SYSID_THISMAV".encode('utf-8')  # Convert to bytes
param_index = -1  # Use -1 to request by name instead of index

conn.mav.param_request_read_send(
    conn.target_system,  # Target system (UAV ID)
    conn.target_component,  # Target component (0 for all)
    param_id,  # Parameter name (max 16 chars)
    param_index  # Set to -1 to use param_name
)

while True:
    msg = conn.recv_match(type='PARAM_VALUE', blocking=True)
    if msg is not None:
        if msg.param_id == "SYSID_THISMAV": 
            print(msg)
            break  # 

PARAM_VALUE {param_id : SYSID_THISMAV, param_value : 1.0, param_type : 4, param_count : 1390, param_index : 65535}


## Set a Parameter value

Send a [PARAM_SET](https://ardupilot.org/dev/docs/mavlink-get-set-params.html)

In [12]:
new_param_value = 2
param_type = 4

conn.mav.param_set_send(
    conn.target_system,  # Target system (UAV ID)
    conn.target_component,  # Target component (0 for all)
    param_id ,  # Parameter name (max 16 chars)
    new_param_value ,
    param_type  # Set to -1 to use param_name
)

#### check the change

In [14]:
conn.mav.param_request_read_send(
    conn.target_system,  # Target system (UAV ID)
    conn.target_component,  # Target component (0 for all)
    param_id,  # Parameter name (max 16 chars)
    param_index  # Set to -1 to use param_name
)

while True:
    msg = conn.recv_match(type='PARAM_VALUE', blocking=True)
    if msg is not None:
        if msg.param_id == "SYSID_THISMAV": 
            print(msg)
            break  # 

PARAM_VALUE {param_id : SYSID_THISMAV, param_value : 2.0, param_type : 4, param_count : 1390, param_index : 65535}
